In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode
from docx import Document
from docx.shared import Inches
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
%%capture
%run "19. CO y CT.ipynb"

In [3]:
Columnas_Cambios = ['CO_Pro_Izq', 
                    'CO_Con_Izq', 
                    'CO_Pro_Der', 
                    'CO_Con_Der', 
                    #'CT_Pro_Izq', 
                    #'CT_Con_Izq', 
                    #'CT_Pro_Der', 
                    #'CT_Con_Der'
                    ]

In [ ]:
Todas_Variables_Significativas = {'Cambios_Promedios_Significativos': {},
                                  'Cambios_Por_Item_Significativos': {},
                                  'Cambios_Filtrados_Significativos': {},
                                  'Cambios_Sumados_Significativos': {},

                                  'Cambios_Tiempo_Promedios_Significativos': {},
                                  'Cambios_Tiempo_Por_Item_Significativos': {},
                                  'Cambios_Tiempo_Filtrados_Significativos': {},
                                  'Cambios_Tiempo_Sumados_Significativos': {}}

In [ ]:
for Nombre_df, df in dfs_Finales.items():

    # ---Preparación de datos---.

    # Seleccionar columnas de interés.
    df_Cambios = df[['ID', 'Categoria_PASO_2023'] + Columnas_Cambios]

    # Filtrar filas con categorías válidas y sin valores nulos en las columnas de interés.
    Categorias_Validas = [
        'Left_Wing', 
        'Progressivism', 
        'Centre',
        'Moderate_Right_A', 
        'Moderate_Right_B',
        'Right_Wing_Libertarian'
    ]

    df_Cambios = df_Cambios[df_Cambios['Categoria_PASO_2023'].isin(Categorias_Validas)].dropna(subset=Columnas_Cambios)


    # ---Estadísticas descriptivas---.

    # Importar Kruskal-Wallis.
    from scipy.stats import kruskal

    # Crear diccionario para almacenar resultados.
    Resultados_Kruskal = {}

    # Agrupar por categoría para cada columna y aplicar Kruskal-Wallis.
    for Nombre_Columna in Columnas_Cambios:
        Grupos = [
            df_Cambios[df_Cambios['Categoria_PASO_2023'] == Categoria][Nombre_Columna]
            for Categoria in Categorias_Validas
        ]

        Estadistico, P_Valor = kruskal(*Grupos)
        Resultados_Kruskal[Nombre_Columna] = {
            'Estadístico': Estadistico,
            'Valor_p': P_Valor
        }

    # Convertir los resultados a DataFrame.
    Tabla_Resultados_Kruskal = pd.DataFrame.from_dict(
        Resultados_Kruskal, orient='index'
    ).reset_index().rename(columns={
        'index': 'Variable',
        'Estadístico': 'Estadistico_Kruskal',
        'Valor_p': 'Valor_p'
    })

    # Ordenar por p-valor ascendente.
    Tabla_Resultados_Kruskal = Tabla_Resultados_Kruskal.sort_values(
        by='Valor_p'
    ).reset_index(drop=True)

    # Filtrar variables significativas (p-valor < 0.05).
    Variables_Significativas = Tabla_Resultados_Kruskal[
        Tabla_Resultados_Kruskal['Valor_p'] < 0.05
        ]['Variable'].tolist()
    
    Todas_Variables_Significativas['Cambios_Promedios_Significativos'][Nombre_df] = Variables_Significativas

    # Mostrar tabla.
    print(Tabla_Resultados_Kruskal)